# Markdown Source

In [1]:
%%writefile tests/demo.md
---
amsthm:
  plain:
  - Theorem
  plain-unnumbered:
  - Lemma
  - Proposition
  - Corollary
  definition:
  - Definition
  - Conjecture
  - Example
  - Postulate
  - Problem
  remark:
  - Case
  remark-unnumbered:
  - Remark
  - Note
  proof:
  - proof
  parentcounter:
  - chapter
...

<div class="Theorem">

$$E=mc^2$$

</div>

<div class="proof">

$$a^n+b^n=c^n$$

What, you don't know how to prove this?

</div>

Overwriting tests/demo.md


## Shorter Markdown Source

In [2]:
%%writefile tests/demo.md
---
amsthm:
  1level2:
  - 1level3
  - 2level3
  2level2:
  - 3level3
  - 4level3
2level1:
  3level3
...


Overwriting tests/demo.md


## In native

In [3]:
!pandoc -s -t native tests/demo.md

Pandoc (Meta {unMeta = fromList [("2level1",MetaInlines [Str "3level3"]),("amsthm",MetaMap (fromList [("1level2",MetaList [MetaInlines [Str "1level3"],MetaInlines [Str "2level3"]]),("2level2",MetaList [MetaInlines [Str "3level3"],MetaInlines [Str "4level3"]])]))]})
[]


## In JSON

In [4]:
!pandoc -s -t json tests/demo.md

{"blocks":[],"pandoc-api-version":[1,17,0,4],"meta":{"amsthm":{"t":"MetaMap","c":{"1level2":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"1level3"}]},{"t":"MetaInlines","c":[{"t":"Str","c":"2level3"}]}]},"2level2":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"3level3"}]},{"t":"MetaInlines","c":[{"t":"Str","c":"4level3"}]}]}}},"2level1":{"t":"MetaInlines","c":[{"t":"Str","c":"3level3"}]}}}

# panflute testing filter

In [5]:
import panflute
panflute.elements.MetaMap?

In [6]:
%%writefile test.py
#!/usr/bin/env python3
"""
Pandoc filter using panflute
"""

import panflute as pf


def prepare(doc):
    pass


def action(elem, doc):
    for i in doc.content:
        print(type(i))
#     if isinstance(elem, pf.MetaMap) and 'amsthm' in elem.content:
#         elem.content['amsthm'].walk(print, doc)

#     if isinstance(elem, pf.MetaMap):
#         print(elem)
#         print(elem.next)
#     if isinstance(elem, pf.MetaMap):
#         print(elem)


def finalize(doc):
    pass


def main(doc=None):
	return pf.run_filter(action,
	                     prepare=prepare,
	                     finalize=finalize,
	                     doc=doc) 


if __name__ == '__main__':
    main()

Overwriting test.py


In [7]:
!chmod +x test.py

## Output

In [8]:
!pandoc -s -t json tests/demo.md | ./test.py

{"pandoc-api-version":[1,17,0,4],"meta":{"amsthm":{"t":"MetaMap","c":{"1level2":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"1level3"}]},{"t":"MetaInlines","c":[{"t":"Str","c":"2level3"}]}]},"2level2":{"t":"MetaList","c":[{"t":"MetaInlines","c":[{"t":"Str","c":"3level3"}]},{"t":"MetaInlines","c":[{"t":"Str","c":"4level3"}]}]}}},"2level1":{"t":"MetaInlines","c":[{"t":"Str","c":"3level3"}]}},"blocks":[]}

# YAML Test

In [13]:
YAML = """amsthm:
amsthm:
  plain:
  - Theorem: Lemma
  - With Space*
  definition:
  - Definition
  remark:
  - Case
  parentcounter:
  - chapter
"""

In [14]:
import yaml

In [15]:
options = yaml.load(YAML)
print(options)

{'amsthm': {'remark': ['Case'], 'parentcounter': ['chapter'], 'definition': ['Definition'], 'plain': [{'Theorem': 'Lemma'}, 'With Space*']}}


In [16]:
options['amsthm']

{'definition': ['Definition'],
 'parentcounter': ['chapter'],
 'plain': [{'Theorem': 'Lemma'}, 'With Space*'],
 'remark': ['Case']}